# Compare `contextlib.ExitStack` with a simple recursive solution for nesting context managers
Motivated by https://news.ycombinator.com/item?id=25041793

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Nesting-multiple-context-managers" data-toc-modified-id="Nesting-multiple-context-managers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Nesting multiple context managers</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Testing</a></span></li><li><span><a href="#Small-number-of-context-managers" data-toc-modified-id="Small-number-of-context-managers-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Small number of context managers</a></span></li><li><span><a href="#Large-number-of-context-managers" data-toc-modified-id="Large-number-of-context-managers-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Large number of context managers</a></span></li></ul></div>

In [1]:
import contextlib
import sys

## Nesting multiple context managers
In principle, one can nest a list of given context managers with a recursive approach:

In [2]:
@contextlib.contextmanager
def multi_cm_recursive(first, *others):
    with first:
        if others:
            with multi_cm_recursive(*others):
                yield
        else:
            yield

A similar effect can be achieved with `contextlib.ExitStack` from the standard library:

In [3]:
@contextlib.contextmanager
def multi_cm_ExitStack(*cms):
    with contextlib.ExitStack() as stack:
        for cm in cms:
            stack.enter_context(cm)
        yield  

## Testing
Implement a function that tests the two approaches with an arbitrary number of context managers:

In [4]:
def test_nested_cms(func, number):
    """
    Arguments:
    * a function that nests context managers
    * the number of context managers that shall be used for testing.
    
    Outputs the number of active context managers before, inside,
    and after the 'with' block, such that the correctness can be
    checked easily.
    """
    print(f"Testing {func.__name__}...")

    ACTIVE_CONTEXT_MANAGERS = 0
    
    def info(where):
        print(f"{ACTIVE_CONTEXT_MANAGERS} context managers active {where} 'with' block")

    @contextlib.contextmanager
    def mock_cm():
        nonlocal ACTIVE_CONTEXT_MANAGERS
        ACTIVE_CONTEXT_MANAGERS += 1
        try:
            yield
        finally:
            ACTIVE_CONTEXT_MANAGERS -= 1
        
    info("before")
    try:
        with func(*(mock_cm() for _ in range(number))):
            info("inside")
    except Exception as error:
        print(f"Exception: {error}", file=sys.stderr)
    info("after")
    print()

## Small number of context managers
For 10 nested context managers, both approaches work well:

In [5]:
test_nested_cms(multi_cm_recursive, 10)
test_nested_cms(multi_cm_ExitStack, 10)

Testing multi_cm_recursive...
0 context managers active before 'with' block
10 context managers active inside 'with' block
0 context managers active after 'with' block

Testing multi_cm_ExitStack...
0 context managers active before 'with' block
10 context managers active inside 'with' block
0 context managers active after 'with' block



## Large number of context managers
The recursive solution exceeds the maximum recursion depth:

In [6]:
test_nested_cms(multi_cm_recursive, 1000)

Testing multi_cm_recursive...
0 context managers active before 'with' block
0 context managers active after 'with' block



Exception: maximum recursion depth exceeded while calling a Python object


`contextlib.ExitStack` works also with many nested context managers:

In [7]:
test_nested_cms(multi_cm_ExitStack, 1000)

Testing multi_cm_ExitStack...
0 context managers active before 'with' block
1000 context managers active inside 'with' block
0 context managers active after 'with' block

